<a href="https://colab.research.google.com/github/Az-Ks/AirQo-Ugandan-Air-Quality-Forecast/blob/master/_submission2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install catboost

     |████████████████████████████████| 64.8MB 62kB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split


from scipy.sparse import csr_matrix
import gc

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,BaggingRegressor,AdaBoostRegressor,ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
#from ngboost import NGBRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold
import datetime
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from fastai.tabular import *
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "rf.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path =  '/content/drive/My Drive/'

# THIS IS THE NEW DATA  GENERATED IN NEW_DATA NOTEBOOK 

In [0]:
train = pd.read_csv(path+'train_df.csv')
test = pd.read_csv(path+'test_df.csv')
train.shape,test.shape

((15539, 3087), (5035, 3086))

In [0]:
temp = ["temp{}".format(i) for i in range(121)]
precip = ["precip{}".format(i) for i in range(121)]
rel_humidity = ["rel_humidity{}".format(i) for i in range(121)]
wind_dir = ["wind_dir{}".format(i) for i in range(121)]
wind_spd = ["wind_spd{}".format(i) for i in range(121)]
atmos_press = ["atmos_press{}".format(i) for i in range(121)]

In [0]:
grp_data11_col = pd.read_csv(path+'grp_data11_col.csv')
grp_data11_col = list(grp_data11_col['0'].values)

grp_data21_col = pd.read_csv(path+'grp_data21_col.csv')
grp_data21_col = list(grp_data21_col['0'].values)

grp_data31_col = pd.read_csv(path+'grp_data31_col.csv')
grp_data31_col = list(grp_data31_col['0'].values)

grp_data41_col = pd.read_csv(path+'grp_data41_col.csv')
grp_data41_col = list(grp_data41_col['0'].values)

grp_data51_col = pd.read_csv(path+'grp_data51_col.csv')
grp_data51_col = list(grp_data51_col['0'].values)

rem_org = pd.read_csv(path+'rem_org.csv')
rem_org = list(rem_org['0'].values)

In [0]:
len(rem_org),len(grp_data11_col),len(grp_data21_col),len(grp_data31_col),len(grp_data41_col),len(grp_data51_col)

(116, 511, 409, 817, 307, 205)

In [0]:
f1 = list(set(temp + precip + rel_humidity + wind_dir + wind_spd + atmos_press + rem_org)) # orig
f2 = list(set(f1 + grp_data11_col)) #day
f3 = list(set(f1 + grp_data21_col)) #typeofday
f4 = list(set(f1 + grp_data31_col)) #type_of_day1
f5 = list(set(f1 + grp_data41_col)) #type_of_day2
f6 = list(set(f1 + grp_data51_col)) #type_of_day3
f7 = list(set(f1 + f3 + f5)) #type_of_day day_type_of_day2
f8 = list(set(f1 + f3 + f6)) #type_of_day day_type_of_day3
f9 = list(set(f1 + f4 + f5)) #type_of_day1 day_type_of_day2
f10 = list(set(f1 + f4 + f6)) #type_of_day1 day_type_of_day3
f11 = list(set(f1 + f5 + f6)) #type_of_day2 day_type_of_day3
f12 = list(set(f1 + f3+ f5 + f6)) #type_of_day type_of_day2 day_type_of_day3
f13 = list(set(f1 + f4+ f5 + f6)) #type_of_day1 type_of_day2 day_type_of_day3
f14 = list(set(f1 + f4+ f5 + f3 + f6)) #type_of_day1 type_of_day2 day_type_of_day3


In [0]:
train = train[f14+['target']]
test = test[f14]
train.shape,test.shape

((15539, 2577), (5035, 2576))

In [0]:
train_id = train['ID']
test_id = test['ID']
y = train['target']

train.drop(['ID','target'],axis=1,inplace=True)
test.drop('ID',axis=1,inplace=True)
train.shape,test.shape

((15539, 2575), (5035, 2575))

In [0]:
X  = train.copy()
Xtest = test.copy()

In [0]:
forecastwindte=pd.read_csv('/content/winddirforecasttest.csv')
forecastwindtr=pd.read_csv('/content/winddirforecasttrain.csv')
for i in forecastwindte :
  X[i]=forecastwindtr[i]
  Xtest[i]=forecastwindte[i]

In [0]:
%%time
from lightgbm import LGBMRegressor
lgb = LGBMRegressor()
lgb.fit(X,y)

imp = pd.DataFrame(lgb.feature_importances_,index=X.columns)
l = list(imp[imp[0]>3].index)
len(l)

CPU times: user 1min 47s, sys: 639 ms, total: 1min 47s
Wall time: 56.5 s


In [0]:
len(l),X.shape

(190, (15539, 2593))

In [0]:
Xtest[l].head()

,wind_dir115,p20_wind_dir,std_wind_dir,wind_dir120,wind_dir_percentile_95Third,p90_wind_dir,atmos_press86,p80_temp,temp59,p20_temp,wind_dir_percentile_80Second,p70_temp,max_wind_dir,temp113,var_precip,rel_humidity58,p1_temp,var_temp,temp49,p80_rel_humidity,var_wind_spd,ptp_wind_dir,mean_wind_spd,rel_humidity_stdMor1,p95_rel_humidity,p99_temp,precip10,wind_dir_percentile_90Eve2,wind_dir_stdMor1,rel_humidity_percentile_20F_half,wind_spd_stdF_half,temp_maxNig2,temp32,wind_dir_percentile_50Noo1,wind_dir_meanNoon,rel_humidity_stdEve1,rel_humidity86,p95_wind_spd,rel_humidity_minNight,p80_wind_spd,...,p20_rel_humidity,wind_spd_meanNoo2,ptp_atmos_press,wind_dir_percentile_10Noon,rel_humidity4,p5_wind_spd,temp_percentile_95Third,wind_dir_percentile_20Noo1,min_wind_dir,std_atmos_press,atmos_press_stdSecond,wind_spd_percentile_95S_half,temp_stdF_half,wind_spd108,temp_percentile_99F_half,atmos_press_stdS_half,temp_stdMor1,wind_dir_percentile_80Noo1,rel_humidity_percentile_70Mor1,rel_humidity_percentile_95F_half,wind_dir12,wind_spd120,std_rel_humidity,p60_wind_spd,atmos_press_stdMor1,mean_wind_dir,p99_rel_humidity,p95_temp,rel_humidity_percentile_40Mor1,wind_dir_percentile_70Mor1,p99_wind_spd,wind_dir_percentile_80Evening,temp1,p30_rel_humidity,wind_dir_percentile_60Eve1,p30_wind_dir,rel_humidity_percentile_99Noo1,wind_dir_percentile_99Eve2,0,1
0,332.50,167.60,83.90,163.60,332.8,334.2,87.90,24.40,18.61,18.58,325.0,23.12,357.2,24.84,0.77500,0.9920,17.20,7.465,22.73,0.9700,0.1543,350.2,0.8125,0.07074,0.9976,26.39,0.0,327.2,92.94,0.8477,0.3184,25.72,19.56,196.00,251.1,0.05365,0.9860,1.604,0.0000,1.089,...,0.7026,0.7210,0.5470,159.90,0.8477,0.3060,26.38,158.40,7.0940,0.1328,0.11127,1.680,1.588,0.8170,23.33,0.16720,1.319,327.8,0.8555,0.9970,313.20,0.7740,0.12420,0.8726,0.08070,231.1,0.9990,25.90,0.7754,286.5,1.818,316.8,22.89,0.7420,308.20,173.1,0.9956,349.2,197.250658,212.394441
1,16.44,154.60,91.25,111.56,348.2,333.5,88.00,23.66,22.55,19.48,186.9,22.47,351.2,21.06,2.30500,0.7324,17.44,5.242,19.89,0.9220,0.2086,345.0,0.8525,0.10333,0.9937,26.73,0.0,326.5,105.20,0.6740,0.4067,20.08,20.39,170.60,167.2,0.03864,0.8755,1.639,0.8574,1.287,...,0.7360,1.0180,0.6094,138.40,0.7773,0.2825,20.22,158.40,6.2900,0.1138,0.10614,1.133,2.310,0.7715,26.95,0.06610,1.964,182.6,0.9688,0.9970,167.90,0.3118,0.10640,0.9330,0.08800,212.9,0.9985,25.47,0.8560,320.5,1.995,299.8,19.70,0.7730,186.00,169.8,0.8030,346.5,146.588427,165.851197
2,93.60,82.94,98.00,68.40,216.8,313.2,90.56,24.81,18.92,19.66,312.2,23.53,356.5,21.00,0.02708,0.9536,17.39,8.664,21.66,0.9385,0.3247,354.2,0.8030,0.09467,0.9520,29.45,0.0,269.2,68.56,0.8490,0.5103,29.84,20.83,264.80,220.1,0.01112,0.9450,2.008,0.5150,1.162,...,0.7870,0.3606,0.6700,28.02,0.8696,0.2776,28.58,147.60,2.2800,0.1364,0.05792,2.078,2.232,0.2917,28.61,0.14230,2.098,321.0,0.8496,0.9463,21.44,1.3950,0.09644,0.7830,0.07526,167.8,0.9650,28.03,0.7847,135.8,2.719,286.0,25.08,0.8564,264.00,96.6,0.9434,307.0,165.153288,165.153288
3,61.03,122.60,106.90,330.80,335.8,334.8,87.90,24.90,22.16,20.08,171.9,23.95,353.2,20.06,0.10790,0.6875,18.75,6.152,22.81,0.8830,0.1854,353.2,0.7974,0.08870,0.9350,27.06,0.0,328.0,58.75,0.6610,0.3945,21.48,25.90,171.90,176.1,0.05353,0.8450,1.520,0.7690,1.316,...,0.6860,0.7020,0.5186,146.20,0.7217,0.2500,21.48,167.60,0.1259,0.1256,0.08550,1.035,1.925,0.5024,27.17,0.09520,2.000,175.8,0.8496,0.8620,334.80,1.0260,0.10077,0.8784,0.06450,199.5,0.9650,26.72,0.7790,330.5,1.759,192.1,20.61,0.7217,97.75,154.1,0.8570,334.8,268.745697,236.704786
4,278.50,88.70,107.06,28.10,351.5,334.5,90.50,27.67,30.08,19.52,289.0,26.58,357.5,22.66,0.00000,0.5270,17.17,17.110,17.95,0.8794,0.1938,352.0,0.8120,0.03253,0.9116,30.89,0.0,296.2,118.00,0.5440,0.4553,19.83,29.45,108.56,133.2,0.05817,0.7935,1.612,0.7890,1.207,...,0.6010,0.8926,0.6924,78.20,0.8210,0.3500,21.89,89.44,5.5080,0.1611,0.14420,0.857,4.110,0.5950,30.98,0.10486,1.837,132.9,0.9014,0.9043,196.10,0.3250,0.13290,0.6800,0.09686,188.6,0.9277,30.17,0.8794,318.0,1.982,304.8,18.06,0.65

In [0]:
X[l].shape

(15539, 190)

In [0]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X[l].values[train_index], X[l].values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    m2  = CatBoostRegressor(n_estimators=5000,eval_metric='RMSE',learning_rate=0.175, random_seed= 42, use_best_model=True )
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=200)
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(Xtest[l])
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 40.1835183	test: 40.1835183	test1: 41.0688158	best: 41.0688158 (0)	total: 146ms	remaining: 12m 10s
200:	learn: 18.7544911	test: 18.7544911	test1: 25.0459735	best: 25.0459735 (200)	total: 11.5s	remaining: 4m 34s
400:	learn: 14.2876025	test: 14.2876025	test1: 23.9297135	best: 23.9297135 (400)	total: 22.9s	remaining: 4m 22s
600:	learn: 11.4385378	test: 11.4385378	test1: 23.3826366	best: 23.3826366 (600)	total: 34.2s	remaining: 4m 10s
800:	learn: 9.3831113	test: 9.3831113	test1: 23.1237098	best: 23.1191011 (750)	total: 45.6s	remaining: 3m 59s
1000:	learn: 7.8503616	test: 7.8503616	test1: 22.9967621	best: 22.9967621 (1000)	total: 57s	remaining: 3m 47s
1200:	learn: 6.6601525	test: 6.6601525	test1: 22.8396990	best: 22.8287511 (1171)	total: 1m 8s	remaining: 3m 36s
1400:	learn: 5.6481574	test: 5.6481574	test1: 22.7690635	best: 22.7532335 (1374)	total: 1m 19s	remaining: 3m 24s
1600:	learn: 4.8291317	test: 4.8291317	test1: 22.6886953	best: 22.6715717 (1588)	total: 1m 31s	remaining: 3m 1

22.59629645158602

In [0]:
np.mean(errcb2)

22.59629645158602

In [0]:
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [0]:
sub.to_csv('indiandatacat22.81.csv',index=False)

In [0]:
from google.colab import files
files.download("indiandatacat22.81.csv")

## **LGBM**

In [0]:
import lightgbm as lgb
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.09,
    'num_leaves': 200,  
    'max_depth': 9,  
    'min_child_samples': 200,   
    'subsample': 0.7,  
    'subsample_freq': 1,  
    'colsample_bytree': 0.7,  
    'min_child_weight': 0,  
    'subsample_for_bin': 200000,  
    'min_split_gain': 0,  
    'reg_alpha': 0,  
    'reg_lambda': 0,  
    'nthread': -1,
    'verbose': 5,
    'n_estimators':10000,
#     'scale_pos_weight'
    }

model = lgb.LGBMRegressor(**params)
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X[l],y):
    X_train, X_test = X[l].iloc[train_index], X[l].iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = model
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(Xtest[l])
    y_pred_totcb2.append(p2)
np.mean(errcb2)

Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 23.3385	valid_1's rmse: 25.8366
[200]	training's rmse: 20.43	valid_1's rmse: 24.6165
[300]	training's rmse: 18.3035	valid_1's rmse: 24.0312
[400]	training's rmse: 16.7081	valid_1's rmse: 23.6337
[500]	training's rmse: 15.2208	valid_1's rmse: 23.3187
[600]	training's rmse: 14.0633	valid_1's rmse: 23.168
[700]	training's rmse: 13.0215	valid_1's rmse: 22.9711
[800]	training's rmse: 12.1298	valid_1's rmse: 22.7497
[900]	training's rmse: 11.2788	valid_1's rmse: 22.6532
[1000]	training's rmse: 10.5641	valid_1's rmse: 22.5508
[1100]	training's rmse: 9.88104	valid_1's rmse: 22.5207
[1200]	training's rmse: 9.30002	valid_1's rmse: 22.4539
[1300]	training's rmse: 8.77031	valid_1's rmse: 22.437
[1400]	training's rmse: 8.29685	valid_1's rmse: 22.4088
[1500]	training's rmse: 7.81564	valid_1's rmse: 22.3896
[1600]	training's rmse: 7.39485	valid_1's rmse: 22.3342
[1700]	training's rmse: 7.02391	valid_1's rmse: 22.322

22.896670261087763

In [0]:
np.mean(errcb2)
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub2 = pd.DataFrame(data=d)
sub2 = sub2[['ID', 'target']]
sub2.target.min(),sub.target.max()

(8.845006924682506, 268.56436092746134)

In [0]:
sub2.to_csv("sub2ind.csv",index  = False)
from google.colab import files
files.download("sub2ind.csv")

## **BLENDING**

In [0]:
submission2 =sub2.copy()
submission2['target']=sub2['target']*0.3+sub['target']*0.7

In [0]:
submission2.to_csv("submission2.csv",index  = False)
from google.colab import files
files.download("submission2.csv")